In [16]:
# from sqlalchemy import create_engine
import sqlalchemy as db

In [17]:
#Engine: Esse objeto atua como uma fonte central de conexões para um banco de dados específico
# usaremos um banco de dados SQLite somente na memória. 

# Como localizamos o banco de dados? Nesse caso, nossa URL inclui a frase /:memory:, 
# que é um indicador para o sqlite3módulo de que usaremos um banco de dados somente na memória . 
# Este tipo de banco de dados é perfeito para experimentar, pois não requer nenhum servidor nem a 
# criação de novos arquivos.
engine = db.create_engine("sqlite:///bancofilmes.db", echo=True)
# echo:para vermos o log sql

In [18]:
conn = engine.connect() 

In [85]:
metadata = db.MetaData()

In [20]:
BDFilmes = db.Table('Filmes', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Name', db.String(255), nullable=False),
              db.Column('Rating', db.Float()),
              db.Column('Ano', db.Integer())
              )
metadata.create_all(engine) 

2023-05-25 16:38:18,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 16:38:18,115 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Filmes")
2023-05-25 16:38:18,119 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 16:38:18,131 INFO sqlalchemy.engine.Engine COMMIT


# Fazendo operações no banco

Adicionando uma linha

In [22]:
inserir = db.insert(BDFilmes).values(Id=1, Name='O homem que desafiou o diabo', Rating=4.5, Ano=2010)

In [24]:
resultado = conn.execute(inserir)

2023-05-25 16:38:50,426 INFO sqlalchemy.engine.Engine INSERT INTO "Filmes" ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 16:38:50,428 INFO sqlalchemy.engine.Engine [generated in 0.00226s] (1, 'O homem que desafiou o diabo', 4.5, 2010)
2023-05-25 16:38:50,433 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
#olhando todas as linhas da tabela
output = conn.execute(BDFilmes.select()).fetchall()
print(output)

2023-05-25 16:39:19,578 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes"
2023-05-25 16:39:19,579 INFO sqlalchemy.engine.Engine [generated in 0.00176s] ()
[(1, 'O homem que desafiou o diabo', 4.5, 2010)]


Adicionando várias linhas

In [26]:
query = db.insert(BDFilmes)
# lista com varias linhas com as chaves e valores a adicionar
valores = [{'Id':2, 'Name':'As Branquelas', 'Rating':2.5, 'Ano':2000},
           {'Id':3, 'Name':'Conexão Jamaica', 'Rating':2.1, 'Ano':2000}
           ]
resultado = conn.execute(query,valores)

2023-05-25 16:43:24,424 INFO sqlalchemy.engine.Engine INSERT INTO "Filmes" ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 16:43:24,429 INFO sqlalchemy.engine.Engine [generated in 0.00662s] ((2, 'As Branquelas', 2.5, 2000), (3, 'Conexão Jamaica', 2.1, 2000))
2023-05-25 16:43:24,442 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
output = conn.execute(db.select([BDFilmes])).fetchall()
print(output)

2023-05-25 16:43:37,857 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes"
2023-05-25 16:43:37,860 INFO sqlalchemy.engine.Engine [cached since 258.3s ago] ()
[(1, 'O homem que desafiou o diabo', 4.5, 2010), (2, 'As Branquelas', 2.5, 2000), (3, 'Conexão Jamaica', 2.1, 2000)]


## Realizando consultas

In [32]:
# atencao, o nome da tabela é filmes, BDFilmes é apenas o objeto da tabela
output = conn.execute('SELECT * FROM Filmes')
print(output.fetchall())

2023-05-25 16:49:49,341 INFO sqlalchemy.engine.Engine SELECT * FROM Filmes
2023-05-25 16:49:49,343 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1, 'O homem que desafiou o diabo', 4.5, 2010), (2, 'As Branquelas', 2.5, 2000), (3, 'Conexão Jamaica', 2.1, 2000)]


In [33]:
output = conn.execute("SELECT Name, Rating FROM Filmes WHERE Ano = 2000")
print(output.fetchall())

2023-05-25 16:51:57,087 INFO sqlalchemy.engine.Engine SELECT Name, Rating FROM Filmes WHERE Ano = 2000
2023-05-25 16:51:57,089 INFO sqlalchemy.engine.Engine [raw sql] ()
[('As Branquelas', 2.5), ('Conexão Jamaica', 2.1)]


## Fazendo consultas diretamente com a API do SQLAlchemy

In [34]:
query = BDFilmes.select().where(BDFilmes.columns.Ano == '2000')
output = conn.execute(query)
print(output.fetchall())

2023-05-25 16:55:33,616 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" = ?
2023-05-25 16:55:33,619 INFO sqlalchemy.engine.Engine [generated in 0.00272s] ('2000',)
[(2, 'As Branquelas', 2.5, 2000), (3, 'Conexão Jamaica', 2.1, 2000)]


In [45]:
# a virgula entende-se como o AND
query = BDFilmes.select().where(db.and_(BDFilmes.columns.Ano == '2000', BDFilmes.columns.Rating<=2.3))
output = conn.execute(query)
print(output.fetchall())

2023-05-25 17:01:11,238 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" = ? AND "Filmes"."Rating" <= ?
2023-05-25 17:01:11,240 INFO sqlalchemy.engine.Engine [cached since 217.5s ago] ('2000', 2.3)
[(3, 'Conexão Jamaica', 2.1, 2000)]


## E se eu quiser usar o pandas?

In [46]:
import pandas as pd

In [48]:
query = BDFilmes.select().where(BDFilmes.columns.Ano.in_([2000]))
output = conn.execute(query)
results = output.fetchall()
results

2023-05-25 17:02:36,451 INFO sqlalchemy.engine.Engine SELECT "Filmes"."Id", "Filmes"."Name", "Filmes"."Rating", "Filmes"."Ano" 
FROM "Filmes" 
WHERE "Filmes"."Ano" IN (?)
2023-05-25 17:02:36,452 INFO sqlalchemy.engine.Engine [generated in 0.00221s] (2000,)


[(2, 'As Branquelas', 2.5, 2000), (3, 'Conexão Jamaica', 2.1, 2000)]

In [50]:
dados = pd.DataFrame(results)

In [51]:
dados.columns = results[0].keys()
dados

,Id,Name,Rating,Ano
0,2,As Branquelas,2.5,2000
1,3,Conexão Jamaica,2.1,2000


In [52]:
# convertendo em csv
dados.to_csv('bancofilmesv2.csv', index=False, sep=';')

# Convertendo um dataframe para tabela sql!

In [53]:
df = pd.read_csv('bancofilmesv2.csv', sep=';')

In [54]:
df.to_sql(con=engine, name='filmesv2', if_exists='replace', index=False)

2023-05-25 17:13:40,299 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("filmesv2")
2023-05-25 17:13:40,301 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 17:13:40,304 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("filmesv2")
2023-05-25 17:13:40,306 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 17:13:40,311 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 17:13:40,313 INFO sqlalchemy.engine.Engine 
CREATE TABLE filmesv2 (
	"Id" BIGINT, 
	"Name" TEXT, 
	"Rating" FLOAT, 
	"Ano" BIGINT
)


2023-05-25 17:13:40,315 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2023-05-25 17:13:40,323 INFO sqlalchemy.engine.Engine COMMIT
2023-05-25 17:13:40,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 17:13:40,330 INFO sqlalchemy.engine.Engine INSERT INTO filmesv2 ("Id", "Name", "Rating", "Ano") VALUES (?, ?, ?, ?)
2023-05-25 17:13:40,332 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ((2, 'As Branquelas', 2.5, 2000), (3, 'Conexão Jamaica', 2.1

In [55]:
conn2 = engine.connect()
metadata2 = db.MetaData()
bdfilmes2 = db.Table('filmesv2', metadata2, autoload=True, autoload_with=engine)

2023-05-25 17:15:03,111 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("filmesv2")
2023-05-25 17:15:03,114 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 17:15:03,118 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-25 17:15:03,121 INFO sqlalchemy.engine.Engine [raw sql] ('filmesv2',)
2023-05-25 17:15:03,124 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("filmesv2")
2023-05-25 17:15:03,126 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 17:15:03,127 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("filmesv2")
2023-05-25 17:15:03,129 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 17:15:03,130 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-25 17:15:03,131 INFO sqlalchemy.engine.Engine [raw sql] ('filmesv2',)
202

In [56]:
query = bdfilmes2.select()
exe = conn.execute(query)
result = exe.fetchmany(5)
for r in result:
    print(r)

2023-05-25 17:15:49,763 INFO sqlalchemy.engine.Engine SELECT filmesv2."Id", filmesv2."Name", filmesv2."Rating", filmesv2."Ano" 
FROM filmesv2
2023-05-25 17:15:49,765 INFO sqlalchemy.engine.Engine [generated in 0.00170s] ()
(2, 'As Branquelas', 2.5, 2000)
(3, 'Conexão Jamaica', 2.1, 2000)


In [57]:
metadata2.create_all(engine) 

2023-05-25 17:17:11,549 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 17:17:11,551 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("filmesv2")
2023-05-25 17:17:11,553 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 17:17:11,556 INFO sqlalchemy.engine.Engine COMMIT


# Ultima forma de criar um banco

In [105]:
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base


In [121]:
engine = db.create_engine("sqlite:///..//impacta.db", echo=True)

In [122]:
conn = engine.connect() 

In [123]:
Base = declarative_base()


class School(Base):

    __tablename__ = "impacta"

    id = Column(Integer, primary_key=True)
    name = Column(String)  

    def __init__(self, name):

        self.name = name    


Base.metadata.create_all(engine)

2023-05-25 18:28:18,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-25 18:28:18,047 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("impacta")
2023-05-25 18:28:18,049 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-25 18:28:18,057 INFO sqlalchemy.engine.Engine COMMIT


In [127]:
impacta2 = db.Table('impacta', metadata, autoload=True, autoload_with=engine)
query = db.insert(impacta2)
# lista com varias linhas com as chaves e valores a adicionar
valores = [{'id':1, 'name':'Bruno'},
           {'id':2, 'name':'Alfredo'}
           ]
resultado = conn.execute(query,valores)

2023-05-25 18:30:48,692 INFO sqlalchemy.engine.Engine INSERT INTO impacta (id, name) VALUES (?, ?)
2023-05-25 18:30:48,694 INFO sqlalchemy.engine.Engine [generated in 0.00213s] ((1, 'Bruno'), (2, 'Alfredo'))
2023-05-25 18:30:48,736 INFO sqlalchemy.engine.Engine COMMIT


In [128]:

query = impacta2.update().values(id = 2).where(impacta2.columns.name == "Bruno")
results = conn.execute(query)

2023-05-25 18:30:51,578 INFO sqlalchemy.engine.Engine UPDATE impacta SET id=? WHERE impacta.name = ?
2023-05-25 18:30:51,580 INFO sqlalchemy.engine.Engine [cached since 152.5s ago] (2, 'Bruno')
2023-05-25 18:30:51,584 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: impacta.id
[SQL: UPDATE impacta SET id=? WHERE impacta.name = ?]
[parameters: (2, 'Bruno')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [130]:
output = conn.execute(impacta2.select()).fetchall()
data = pd.DataFrame(output)
data.columns = output[0].keys()
data

2023-05-25 18:30:55,432 INFO sqlalchemy.engine.Engine SELECT impacta.id, impacta.name 
FROM impacta
2023-05-25 18:30:55,435 INFO sqlalchemy.engine.Engine [cached since 155.6s ago] ()


,id,name
0,1,Bruno
1,2,Alfredo


In [131]:
output = conn.execute('SELECT * FROM impacta')
print(output.fetchall())

2023-05-25 18:30:59,821 INFO sqlalchemy.engine.Engine SELECT * FROM impacta
2023-05-25 18:30:59,825 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1, 'Bruno'), (2, 'Alfredo')]
